In [7]:
import keras
import cv2
import os
import glob
import random
import numpy as np
import pandas as pd

In [2]:
# load saliency and motion videos and split them into chunks
split = 1 # split the video every five seconds
path = '../'

In [31]:
def splitVideo(videos, split, tp):
    newdir = path+'datasets/content/'+tp+'splitby-'+str(split)+'-seconds'
    if os.path.exists(newdir):
        os.system('rm -rf '+newdir)
    os.makedirs(newdir)
    for video in videos:
        command = 'ffmpeg -i ../datasets/content/'+tp+'/'+video+' -reset_timestamps 1 -map 0 -segment_time '+str(split)+' -f segment '+newdir+'/'+video[:-4]+'%03d.mp4'
        os.system(command)

In [138]:
def extractImages(videos, split, tp):
    newdir = path+'datasets/content/'+tp+'splitby-'+str(split)+'-seconds'
    if os.path.exists(newdir):
        os.system('rm -rf '+newdir)
    os.makedirs(newdir)
    for video in videos:
        os.system('ffmpeg -i ../datasets/content/'+tp+'/'+video+' -vf fps=30 '+newdir+'/'+video[:-4]+'%d.png')

In [143]:
saliVideos = os.listdir(path+'datasets/content/saliency/')
motionVideos = os.listdir(path+'datasets/content/motion/')

extractImages(saliVideos, split, 'saliency')
extractImages(motionVideos, split, 'motion')

In [145]:
# make both saliency and motion map videos as numpy arrays
def makeVideoNumpy(videos, split, tp):
    fps = 30
    totalFrames = 1800
    imagesToRead = fps*split # total images to read based on split
    newdir = path+'datasets/content/'+tp+'Images'
    for video in videos:
        count = 1
        for i in range(1, totalFrames+1, imagesToRead):
            npArray = newdir+'/'+video[:-4]+str(count)+'.npy'
            count += 1
            tempArray = []
            for j in range(imagesToRead):
                imageName = newdir+'/'+video[:-4]+str(i+j)+'.png'
                image = cv2.imread(imageName, 0)
                tempArray.append(image)
            #print(np.array(tempArray).shape)
            np.save(npArray, np.array(tempArray))
makeVideoNumpy(saliVideos, split, 'saliency')
makeVideoNumpy(motionVideos, split, 'motion')

In [133]:
# make tiles information as numpy arrays
def makeTilesNumpyBackup(videosTileData, split):
    files = glob.glob('../datasets/sensory/tile/*01*.csv')
    files = sorted(files)[5:]
    fps = 30
    totalFrames = 1800
    imagesToRead = fps*split # total images to read based on split
    for video, f in zip(videosTileData, files):
        count = 1
        for i in range(1, totalFrames+1, imagesToRead):
            npArray = f[:-8]+str(count)+'_split_by_'+str(split)+'_seconds.npy'
            tempArray = []
            for j in range(imagesToRead):
                tempArray.append(video[i+j-1])
            #print(np.array(tempArray).shape)
            np.save(npArray, np.array(tempArray))
            count += 1

In [3]:
# make tiles information as numpy arrays
def makeTilesNumpy(video, f, split):
    fps = 30
    totalFrames = 1800
    imagesToRead = fps*split # total images to read based on split
    count = 1
    for i in range(1, totalFrames+1, imagesToRead):
        npArray = f[:-8]+str(count)+'.npy'
        tempArray = []
        for j in range(imagesToRead):
            tempArray.append(video[i+j-1])
        #print(np.array(tempArray).shape)
        np.save(npArray, np.array(tempArray))
        count += 1

In [4]:
# encode tile information into images in banary fashion for convenience
def encodeTileInfoToImages():
    files = glob.glob('../datasets/sensory/tile/*.csv')
    for f in files:
        width = 240
        breadth = 120
        tileSize = 12
        frames = []
        tilesInColumn = width / tileSize
        lines = open(f, 'r')
        lines = lines.readlines()[1:]
        tilesInFrames = []
        for line in lines:
            line = line.strip().split(',')[1:]
            line = [int(i) for i in line]
            tilesInFrames.append(line)
        for i, tiles in enumerate(tilesInFrames):
            frame = np.zeros(width*breadth, dtype=bool)
            for tileNo in tiles:
                tileRowNumber = int((tileNo - 1) / tilesInColumn)
                tileColumnNumber = (tileNo - 1) % tilesInColumn
                firstPixel = tileRowNumber * width * tileSize + tileColumnNumber * tileSize
                for rowPixel in range(0, tileSize):
                    for columnPixel in range(0, tileSize):
                        frame[int(firstPixel + rowPixel * width + columnPixel)] = 1
            frame = frame.reshape((120, 240))
            frames.append(frame)
        makeTilesNumpy(frames, f, split)
        frames = None
encodeTileInfoToImages()

In [5]:
files = glob.glob('../datasets/sensory/tile/*.csv')
print(len(files))

500


In [28]:
p ='../datasets/content/saliencyImages/landscape_saliency1.npy'
data = np.load(p)
print(data.shape)

(150, 1920, 3840)


In [50]:
width = 512
breadth = 512
tiles = [1, 14, 19, 12]
tileSize = 64
frame = np.zeros(width * breadth)
tilesInColumn = width / tileSize

for tileNo in tiles:
    tileRowNumber = int((tileNo - 1) / tilesInColumn)
    tileColumnNumber = (tileNo - 1) % tilesInColumn
    firstPixel = tileRowNumber * width * tileSize + tileColumnNumber * tileSize
    for rowPixel in range(0, tileSize):
        for columnPixel in range(0, tileSize):
            frame[int(firstPixel + rowPixel * width + columnPixel)] = 255
frame = np.reshape(frame, (breadth, width))width*
print(frame)

[[255. 255. 255. ...   0.   0.   0.]
 [255. 255. 255. ...   0.   0.   0.]
 [255. 255. 255. ...   0.   0.   0.]
 ...
 [  0.   0.   0. ...   0.   0.   0.]
 [  0.   0.   0. ...   0.   0.   0.]
 [  0.   0.   0. ...   0.   0.   0.]]


In [ ]:
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt

plt.imshow(frame, interpolation='nearest')
plt.savefig('sample.png')

In [20]:
def makeTilesProbNumpy(video, f, split):
    fps = 30
    totalFrames = 1800
    imagesToRead = fps*split # total images to read based on split
    count = 1
    for i in range(1, totalFrames+1, imagesToRead):
        npArray = '../datasets/sensory/tileProb/'+f[25:-8]+str(count)+'.npy'
        tempArray = []
        for j in range(imagesToRead):
            tempArray.append(video[i+j-1])
        #print(np.array(tempArray).shape)
        np.save(npArray, np.array(tempArray))
        count += 1

In [21]:
files = glob.glob('../datasets/sensory/tile/*.csv')
newdir = '../datasets/sensory/tileProb'
if os.path.exists(newdir):
    os.system('rm -rf '+newdir)
os.makedirs(newdir)
for f in files:
    lines = open(f, 'r')
    lines = lines.readlines()[1:]
    frames = np.zeros((1800, 200))
    count = 0
    for line in lines:
        line = line.strip().split(',')[1:]
        line = [int(i) for i in line]
        for i in line:
            frames[count][i-1] = 1
        count += 1
    makeTilesProbNumpy(frames, f, 1)

In [14]:
f = '../datasets/sensory/tile/ride_user44_tile.csv'
print('../datasets/sensory/tileProb/'+f[25:-8]+str(count)+'.npy')

../datasets/sensory/tileProb/ride_user44_1.npy


In [28]:
## Separate the data into Train and Test folders
files = glob.glob('../datasets/sensory/tile/*.csv')
random.shuffle(files)

In [34]:
train = files[:400]
test = files[400:]
for i in test:
    for j in range(1, 61):
        os.system('mv '+i[:-8]+str(j)+'.npy '+'../datasets/testData/sens/')